In [8]:
## So first let's import the data and set up our libraries 
import pandas as pd 
wm_data = pd.read_excel("Weights-Measures_Data.xlsx")

In [3]:
##Great now let's take a peek under the hood of the data 
wm_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12755 entries, 0 to 12754
Data columns (total 35 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Certificate of Inspection      12755 non-null  object        
 1   Inspection Number              12755 non-null  object        
 2   Date of Occurance              12755 non-null  datetime64[ns]
 3   Year (added)                   12755 non-null  int64         
 4   Business Unique ID             12750 non-null  object        
 5   Business Name                  12755 non-null  object        
 6   DBA Trade Name                 3552 non-null   object        
 7   Business Category              12755 non-null  object        
 8   Inspection type                12755 non-null  object        
 9   Device Category                12755 non-null  object        
 10  Device Class                   12699 non-null  object        
 11  Device Type    

In [24]:
## First we want to verify if the Inspection Number values are unique
wm_data['Inspection Number'].is_unique
pd.set_option('display.max_columns', None)  
wm_data[wm_data['Inspection Number'].duplicated(keep=False)]

,Certificate of Inspection,Inspection Number,Date of Occurance,Year (added),Business Unique ID,Business Name,DBA Trade Name,Business Category,Inspection type,Device Category,Device Class,Device Type,Devices Inspected,Devices Approved,Devices Condemned/Confiscated,Confiscated,Address Type,Building Number,Street Name,Street Name 2,Unit Type,Unit,City,State,Postcode,Borough,Community Board,Council District,BIN,BBL,NTA,Latitude,Longitude,,.1
5,A0156217,INS-000720678,2024-02-28,2024,BA-1559330-2022,THE HEIGHTS MARKET CORP,NaN,Grocery-Retail - 808,Scale,Scale,Scale Inspection,Computing Scale - 01,4,4,0,NaN,Complete Address,1520,SAINT NICHOLAS AVE,NaN,NaN,NaN,NEW YORK,NY,10033,Manhattan,112.0,10.0,1063788.0,1.021570e+09,MN35,40.852479,-73.931411,NaN,NaN
6,A0156217,INS-000720678,2024-02-28,2024,BA-1559330-2022,THE HEIGHTS MARKET CORP,NaN,Grocery-Retail - 808,Scale,Scale,Scale Inspection,Pre-pack Scale - 01,1,1,0,NaN,Complete Address,1520,SAINT NICHOLAS AVE,NaN,NaN,NaN,NEW YORK,NY,10033,Manhattan,112.0,10.0,1063788.0,1.021570e+09,MN35,40.852479,-73.931411,NaN,NaN
17,A0156705,INS-000721752,2024-03-04,2024,BA-1088978-2022,A & C SEAFOOD & MEAT MARKET INC.,NaN,Supermarket - 819,Scale,Scale,Scale Inspection,Computing Scale - 01,5,5,0,NaN,Complete Address,1237,AVENUE U,NaN,NaN,NaN,BROOKLYN,NY,11229,Brooklyn,315.0,48.0,3197984.0,3.073170e+09,BK25,40.598558,-73.958956,NaN,NaN
18,A0156705,INS-000721752,2024-03-04,2024,BA-1088978-2022,A & C SEAFOOD & MEAT MARKET INC.,NaN,Supermarket - 819,Scale,Scale,Scale Inspection,Customer Scale - 01,1,1,0,NaN,Complete Address,1237,AVENUE U,NaN,NaN,NaN,BROOKLYN,NY,11229,Brooklyn,315.0,48.0,3197984.0,3.073170e+09,BK25,40.598558,-73.958956,NaN,NaN
19,A0156705,INS-000721752,2024-03-04,2024,BA-1088978-2022,A & C SEAFOOD & MEAT MARKET INC.,NaN,Supermarket - 819,Scale,Scale,Scale Inspection,Pre-pack Scale - 01,6,6,0,NaN,Complete Address,1237,AVENUE U,NaN,NaN,NaN,BROOKLYN,NY,11229,Brooklyn,315.0,48.0,3197984.0,3.073170e+09,BK25,40.598558,-73.958956,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12737,A0205858,INS-000816521,2025-01-02,2025,BA-1726866-2025,Good fortune supermarket,NaN,Supermarket - 819,Scale,Scale,Scale Inspection,Pre-pack Scale - 01,2,2,0,NaN,Complete Address,4276,MAIN ST,NaN,NaN,NaN,FLUSHING,NY,11355,Queens,407.0,20.0,4622994.0,4.051248e+09,QN22,40.753137,-73.827283,NaN,NaN
12740,A0211733,INS-000829532,2025-02-12,2025,BA-1727176-2025,LINA FOOD CORP.,C-Town Supermarket,Supermarket - 819,Scale,Scale,Scale Inspection,Computing Scale - 01,9,9,0,NaN,Complete Address,4918,BROADWAY,NaN,NaN,NaN,NEW YORK,NY,10034,Manhattan,112.0,10.0,1082052.0,1.022350e+09,MN01,40.867432,-73.921919,NaN,NaN
12741,A0214071,INS-000833234,2025-02-28,2025,BA-1336537-2022,GREENBAY ORGANIC FOODS LLC,NaN,Grocery-Retail - 808,Scale,Scale,Scale Inspection,Pre-pack Scale - 01,1,1,0,NaN,Complete Address,3114,30TH AVE,NaN,NaN,NaN,ASTORIA,NY,11102,Queens,401.0,22.0,4008488.0,4.006150e+09,QN70,40.766788,-73.921196,NaN,NaN
12748,A0216338,INS-000837260,2025-03-14,2025,BA-1192106-2022,"CHEF'S CHOICE CASH & CARRY FOOD DISTRIBUTOR, INC.",NaN,Wholesale Food Market - 718,Scale,Scale,Scale Inspection,Computing Scale - 01,4,4,0,NaN,Complete Address,1051,UTICA AVE,NaN,NaN,NaN,BROOKLYN,NY,11203,Brooklyn,317.0,45.0,3104354.0,3.047350e+09,BK96,40.647234,-73.929888,NaN,NaN


In [26]:
## We know that the inspection number values are not unique. There can be one inspection number used for multiple devices.
# Create standardized Full Address
wm_data["Full Address"] = (
    wm_data["Building Number"].fillna('').astype(str).str.strip() + " " +
    wm_data["Street Name"].fillna('').astype(str).str.strip() + ", " +
    wm_data["Postcode"].astype(str)
)

# Drop duplicates based on Inspection Number to avoid double-counting
unique_inspections = wm_data.drop_duplicates(subset=["Inspection Number"])

# Count top addresses by number of unique inspections
top_addresses = unique_inspections["Full Address"].value_counts().head(10)

# Show result
top_addresses

Full Address
245 MESEROLE AVE, 11222           943
 CENTRAL PARK WEST, 10023          68
245 MESEROLE AVENUE, 11222         33
 EAST 17 STREET, 10003             21
SEC COLUMBUS AVENUE, 10024          8
880 GARRISON AVE, 10474             8
1317 OAKPOINT AVE, 10474            7
1 BAY BLVD, 11559                   7
1 JFK INTL AIRPORT BLVD, 11430      7
20 GRAND ARMY PLZ, 11238            7
Name: count, dtype: int64

In [5]:
## Okay...so now let's tackle these 7 key investigative questions: 
## 1. Identify the repeat offenders: businesses with multiple failed inspections or confiscated/condemned devices.
## 2. Identify the geographic hotspots of device violations by borough and postal code 
## 3. Identify the most commonly inspected and/or condemned device type 
## 4. Are there seasonal spikes or trends in failures or enforcement
## 5. What business types are most at risk of violations 
## 6. Which borough and business types received the highest confiscations
## 7. How likely are devices to be condemned based on routine vs. complaint-driven inspections.

In [6]:
## Let's first identify the repeat offenders: businesses with multiple failed inspections or confiscated/condemned devices.
## Let's first filter for failed inspections
## Then we'll count the number of failed inspections and look at the top 10

# Step 1: Filter to rows where any devices were condemned or confiscated
failed_inspections = wm_data[wm_data["Devices Condemned/Confiscated"] > 0]

# Step 2: Drop duplicates by Inspection Number to count each failed inspection only once
unique_failed = failed_inspections.drop_duplicates(subset=["Inspection Number"])

# Step 3: Count how many unique failed inspections each business had
repeat_offenders = unique_failed["Business Name"].value_counts()

# Step 4: Show top 10 repeat offenders
repeat_offenders.head(10)

Business Name
BOLLA OPERATING CORP.              85
SPEEDWAY LLC                       46
AMG RETAIL I LLC                   16
SPRAGUE OPERATING RESOURCES LLC    13
JAG TRANSPORTING INC               12
LOGIC FUEL TRANSPORTATION CORP      8
STAR ENERGY TRANSPORTATION INC.     7
E.S.F. TRANSPORT INC.               6
BERIL CORP                          5
FIVE BORO FUEL OIL LTD              5
Name: count, dtype: int64

In [28]:
## These are the most common device types to fail inspections
# Step 1: Filter rows where devices were condemned or confiscated
failed_inspections = wm_data[wm_data["Devices Condemned/Confiscated"] > 0]

# Step 2: Remove duplicates based on Inspection Number + Device Type
unique_failed_devices = failed_inspections.drop_duplicates(subset=["Inspection Number", "Device Type"])

# Step 3: Count the most common device types to fail
most_common_devices = unique_failed_devices["Device Type"].value_counts().head(10)

# Step 4: Display results
most_common_devices

Device Type
Octane Sample           1135
Petrol Meter Type B       86
Computing Scale - 01      81
Petrol Meter Type A       31
Petrol Pump Blend         14
Pre-pack Scale - 01       14
Petrol Pump Single        12
No Fee Scale              12
Platform Scale - 02        9
Petrol Pump Diesel         8
Name: count, dtype: int64

In [30]:
## let's see the general info 
unique_failed_devices.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1416 entries, 56 to 12731
Data columns (total 36 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Certificate of Inspection      1416 non-null   object        
 1   Inspection Number              1416 non-null   object        
 2   Date of Occurance              1416 non-null   datetime64[ns]
 3   Year (added)                   1416 non-null   int64         
 4   Business Unique ID             1416 non-null   object        
 5   Business Name                  1416 non-null   object        
 6   DBA Trade Name                 525 non-null    object        
 7   Business Category              1416 non-null   object        
 8   Inspection type                1416 non-null   object        
 9   Device Category                1416 non-null   object        
 10  Device Class                   1400 non-null   object        
 11  Device Type         

In [34]:
# Now from our list of top repeat offenders, let's see what business category they fall into
# First, extract the list of top repeat offenders
top_offenders = [
    "BOLLA OPERATING CORP.",
    "SPEEDWAY LLC",
    "AMG RETAIL I LLC",
    "SPRAGUE OPERATING RESOURCES LLC",
    "JAG TRANSPORTING INC",
    "LOGIC FUEL TRANSPORTATION CORP",
    "STAR ENERGY TRANSPORTATION INC.",
    "E.S.F. TRANSPORT INC.",
    "BERIL CORP",
    "FIVE BORO FUEL OIL LTD"
]

# Step 1: Use unique failed inspections (one row per inspection)
unique_failed = wm_data[wm_data["Devices Condemned/Confiscated"] > 0].drop_duplicates(subset=["Inspection Number"])

# Step 2: Filter for just the top repeat offenders
top_failed = unique_failed[unique_failed["Business Name"].isin(top_offenders)]

# Step 3: Check distinct combinations of business name and category
top_failed[["Business Name", "Business Category"]].drop_duplicates()

,Business Name,Business Category
456,SPRAGUE OPERATING RESOURCES LLC,Fuel Oil Dealer - 814
541,BOLLA OPERATING CORP.,Gas Station-Retail - 815
3140,SPEEDWAY LLC,Gas Station-Retail - 815
5234,BERIL CORP,Gas Station-Retail - 815
5253,STAR ENERGY TRANSPORTATION INC.,Fuel Oil Dealer - 814
5337,AMG RETAIL I LLC,Gas Station-Retail - 815
5689,JAG TRANSPORTING INC,Fuel Oil Dealer - 814
5754,LOGIC FUEL TRANSPORTATION CORP,Fuel Oil Dealer - 814
5944,SPRAGUE OPERATING RESOURCES LLC,Gasoline Truck-Retail - 822
6063,E.S.F. TRANSPORT INC.,Fuel Oil Dealer - 814


In [36]:
## Now lets get a ranked list of what devices were being condemned at each repeat offender's inspection
# Use full failed inspections (per device row)
failed_inspections = wm_data[wm_data["Devices Condemned/Confiscated"] > 0]

# Filter to just top offenders
top_failed_devices = failed_inspections[failed_inspections["Business Name"].isin(top_offenders)]

# Count device types condemned at these businesses
device_counts = top_failed_devices["Device Type"].value_counts()

# Display ranked list
device_counts

Device Type
Octane Sample          140
Petrol Meter Type B     49
Petrol Pump Diesel       5
Petrol Pump Blend        4
No Fee Gas Pump          2
Petrol Meter Type A      2
Petrol Pump Single       1
Name: count, dtype: int64

In [40]:
##Now let's see which device type had the most failure per business
# Full failed inspections at device level
failed_inspections = wm_data[wm_data["Devices Condemned/Confiscated"] > 0]

# Filter for top offenders
top_failed_devices = failed_inspections[failed_inspections["Business Name"].isin(top_offenders)]

# Group by Business and Device Type, count failures
device_failures_per_business = top_failed_devices.groupby(
    ["Business Name", "Device Type"]
).size().sort_values(ascending=False)

device_failures_per_business

Business Name                    Device Type        
BOLLA OPERATING CORP.            Octane Sample          82
SPEEDWAY LLC                     Octane Sample          38
AMG RETAIL I LLC                 Octane Sample          16
SPRAGUE OPERATING RESOURCES LLC  Petrol Meter Type B    13
JAG TRANSPORTING INC             Petrol Meter Type B    12
LOGIC FUEL TRANSPORTATION CORP   Petrol Meter Type B     7
STAR ENERGY TRANSPORTATION INC.  Petrol Meter Type B     7
E.S.F. TRANSPORT INC.            Petrol Meter Type B     6
FIVE BORO FUEL OIL LTD           Petrol Meter Type B     4
BERIL CORP                       Octane Sample           4
SPEEDWAY LLC                     Petrol Pump Diesel      3
                                 No Fee Gas Pump         2
BOLLA OPERATING CORP.            Petrol Pump Blend       2
SPEEDWAY LLC                     Petrol Pump Blend       2
BOLLA OPERATING CORP.            Petrol Pump Diesel      1
FIVE BORO FUEL OIL LTD           Petrol Meter Type A     1
LOG

Between 2023 and 2025, Bolla Operating Corp. topped the list of repeat offenders in NYC’s Weights and Measures inspections, failing 82 inspections across multiple locations — more than any other company. Speedway LLC followed with 38 failed inspections, and AMG Retail I LLC with 16.

The violations uncovered go beyond just faulty gas pumps. At the heart of these inspections were two critical issues: fuel quality and metering accuracy — both of which directly impact how much fuel consumers receive and what they pay for.

Bolla alone failed 82 octane sample tests, the highest in the city, suggesting a pattern of mislabeled or tampered fuel — a practice that could mean consumers are paying premium prices for regular-grade gas. In total, the top 10 offenders were cited in over 140 octane sampling failures, pointing to a widespread and systemic issue in fuel integrity across NYC.

At the same time, wholesalers like Sprague Operating Resources and JAG Transporting Inc. were repeatedly cited for Petrol Meter Type B violations — truck-mounted flow meters used to deliver heating oil or diesel to buildings and commercial customers. These meters, when miscalibrated, can result in underdelivery or overbilling, raising serious concerns about fuel delivery fraud in the supply chain.

In short, NYC’s most frequently cited fuel businesses failed not only to dispense fuel accurately — they often failed to deliver what they claimed was in the tank at all. These violations reflect more than broken hardware; they point to systemic negligence — or worse, economic deception — affecting everyday consumers and businesses alike.

In [44]:
## 2. Identify the geographic hotspots of device violations by borough, address, and postal code
filtered_inspections = wm_data[wm_data["Devices Condemned/Confiscated"] > 0]
## Now let's filter the top boroughs 
# Drop duplicate inspections (keep one row per inspection)
unique_inspections = filtered_inspections.drop_duplicates(subset=["Inspection Number"])

# Now count boroughs by unique inspections
borough_violations = unique_inspections["Borough"].value_counts()

borough_violations

Borough
Brooklyn         481
Queens           453
Bronx            256
Staten Island    113
Manhattan         85
Outside NYC        3
Name: count, dtype: int64

In [46]:
## Now let's filter the top zip codes
# Drop duplicate inspections based on Inspection Number
unique_inspections = filtered_inspections.drop_duplicates(subset=["Inspection Number"])

# Count zip codes by unique inspections
zip_violations = unique_inspections["Postcode"].value_counts()

zip_violations.head(10)

Postcode
11222    125
11101     31
11236     30
11434     24
11385     24
11207     22
11234     21
11218     20
10462     20
10314     19
Name: count, dtype: int64

In [52]:
## Now let's filter the top business addresses 
# Create 'Full Address' column
filtered_inspections["Full Address"] = (
    filtered_inspections["Building Number"].fillna('').astype(str).str.strip() + " " +
    filtered_inspections["Street Name"].fillna('').astype(str).str.strip() + ", " +
    filtered_inspections["Postcode"].astype(str)
)

# Drop duplicate inspections by Inspection Number
unique_inspections = filtered_inspections.drop_duplicates(subset=["Inspection Number"])

# Count unique inspections per address
address_violations = unique_inspections["Full Address"].value_counts()

address_violations

/var/folders/6p/9zpd5gjx1dj6zpc834rhv6d40000gn/T/ipykernel_43373/4119333716.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_inspections["Full Address"] = (


Full Address
245 MESEROLE AVE, 11222                109
2985 JEROME AVE, 10468                   4
196-03 GRAND CENTRAL PARKWAY, 11423      4
8521 7TH AVE, 11228                      4
117 MORNINGSIDE AVE, 10027               4
                                      ... 
2317 CONEY ISLAND AVE, 11223             1
10020 METROPOLITAN AVE, 11375            1
1810 FOREST AVE, 10303                   1
1650 FOREST AVE, 10302                   1
14615 UNION TURNPIKE, 11367              1
Name: count, Length: 803, dtype: int64

In [54]:
address_violations.to_csv("address_violations_counts.csv", header=True)

In [56]:
## Our research confirms that the centralized testing station is located at 245 Meserole Avenue, Brooklyn, NY 11222
# Step 1: Exclude centralized testing station at 245 Meserole
remaining_inspections = failed_inspections.copy()
remaining_inspections = remaining_inspections[
    ~(
        (remaining_inspections["Building Number"].astype(str).str.strip() == "245") &
        (remaining_inspections["Street Name"].str.upper().str.contains("MESEROLE"))
    )
]

# Step 2: Create 'Full Address' column
remaining_inspections["Full Address"] = (
    remaining_inspections["Building Number"].fillna('').astype(str).str.strip() + " " +
    remaining_inspections["Street Name"].fillna('').astype(str).str.strip() + ", " +
    remaining_inspections["Postcode"].astype(str)
)

# Step 3: Drop duplicate inspections to count unique inspection events
unique_remaining = remaining_inspections.drop_duplicates(subset=["Inspection Number"])

# Step 4: Count addresses by unique inspections
address_counts = unique_remaining["Full Address"].value_counts()

# Step 5: Display top 10 addresses
address_counts.head(10)

Full Address
1314 SEDGWICK AVE, 10452               4
2173 GRAND CONCOURSE, 10453            4
117 MORNINGSIDE AVE, 10027             4
2985 JEROME AVE, 10468                 4
8521 7TH AVE, 11228                    4
914 REMSEN AVE, 11236                  4
1981 OCEAN AVE, 11230                  4
1982 BRONXDALE AVE, 10462              4
1193 MYRTLE AVE, 11221                 4
196-03 GRAND CENTRAL PARKWAY, 11423    4
Name: count, dtype: int64

In [20]:
## Now let's see the most popular device types from the remaining inspections
device_type_counts = remaining_inspections["Device Type"].value_counts()
device_type_counts.head(10)

Device Type
Octane Sample           1135
Computing Scale - 01      82
Petrol Pump Blend         15
Pre-pack Scale - 01       14
Petrol Pump Single        12
No Fee Scale              12
Petrol Pump Diesel         9
Platform Scale - 02        9
No Fee Gas Pump            7
Petrol Meter Type B        5
Name: count, dtype: int64

In [58]:
## Now let’s identify the top boroughs where violations occurred — excluding inspections conducted at the centralized Meserole testing site.
# Ensure all fields are strings for safe comparison
# Step 1: Prepare fields and exclude Meserole site (your code)
filtered_inspections = failed_inspections.copy()

filtered_inspections["Building Number"] = filtered_inspections["Building Number"].astype(str).str.strip()
filtered_inspections["Street Name"] = filtered_inspections["Street Name"].astype(str).str.strip().str.upper()

remaining_inspections = filtered_inspections[
    ~(
        (filtered_inspections["Building Number"] == "245") &
        (filtered_inspections["Street Name"].str.contains("MESEROLE", na=False))
    )
]

# Step 2: Deduplicate by Inspection Number to count unique inspections
unique_remaining = remaining_inspections.drop_duplicates(subset=["Inspection Number"])

# Step 3: Count borough violations by unique inspections
borough_violations = unique_remaining["Borough"].value_counts()

borough_violations

Borough
Queens           453
Brooklyn         370
Bronx            256
Staten Island    113
Manhattan         85
Outside NYC        3
Name: count, dtype: int64

In [60]:
# Count violations by ZIP/postal code
# Drop duplicate inspections first
unique_remaining = remaining_inspections.drop_duplicates(subset=["Inspection Number"])

# Count by Postcode using unique inspections only
postal_violations = unique_remaining["Postcode"].value_counts()

postal_violations.head(10)

Postcode
11101    31
11236    30
11434    24
11385    24
11207    22
11234    21
11218    20
10462    20
11208    19
10314    19
Name: count, dtype: int64

In [62]:
# Create Full Address column
remaining_inspections["Full Address"] = (
    remaining_inspections["Building Number"].fillna('').astype(str).str.strip() + " " +
    remaining_inspections["Street Name"].fillna('').astype(str).str.strip() + ", " +
    remaining_inspections["Postcode"].astype(str)
)

# Drop duplicates by Inspection Number to count each inspection once
unique_remaining = remaining_inspections.drop_duplicates(subset=["Inspection Number"])

# Count addresses by unique inspections
top_addresses = unique_remaining["Full Address"].value_counts()

top_addresses.head(20)

/var/folders/6p/9zpd5gjx1dj6zpc834rhv6d40000gn/T/ipykernel_43373/1534240882.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  remaining_inspections["Full Address"] = (


Full Address
1314 SEDGWICK AVE, 10452               4
2173 GRAND CONCOURSE, 10453            4
117 MORNINGSIDE AVE, 10027             4
2985 JEROME AVE, 10468                 4
8521 7TH AVE, 11228                    4
914 REMSEN AVE, 11236                  4
1981 OCEAN AVE, 11230                  4
1982 BRONXDALE AVE, 10462              4
1193 MYRTLE AVE, 11221                 4
196-03 GRAND CENTRAL PARKWAY, 11423    4
1317 OAKPOINT AVE, 10474               3
2430 FDR DR, 10010                     3
2001 CROPSEY AVE, 11214                3
3515 GREENPOINT AVE, 11101             3
1152 UTICA AVE, 11203                  3
109 E 138TH ST, 10451                  3
355 1ST AVE, 10010                     3
8404 PARSONS BLVD, 11432               3
3500 CONNER ST, 10475                  3
8505 ASTORIA BLVD, 11370               3
Name: count, dtype: int64

In [63]:
# Check uniqueness in weights and measures dataset
wm_unique = wm_data["Inspection Number"].is_unique
print(f"Inspection Numbers unique in weights_measures dataset? {wm_unique}")

Inspection Numbers unique in weights_measures dataset? False


In [73]:
## Now we're looking to cross-reference inspection numbers with charges by merging both datasets, but we must account for duplicate inspection numbers

# Step 1: Load the datasets
wm_data = pd.read_excel("Weights-Measures_Data.xlsx")
charges_data = pd.read_csv("DCWP_Charges.csv")

# Step 2: Filter to failed inspections only (any device condemned in the inspection)
failed_inspections = wm_data[wm_data["Devices Condemned/Confiscated"] > 0].copy()

# Step 3: Clean and align formats
failed_inspections["Inspection Number"] = failed_inspections["Inspection Number"].astype(str)
charges_data["Inspection Number"] = charges_data["Inspection Number"].astype(str)

# Step 4: Aggregate failed inspections to unique inspections (one row per inspection)
failed_inspections_agg = (
    failed_inspections
    .groupby("Inspection Number", as_index=False)
    .agg({
        "Devices Condemned/Confiscated": "sum",
        # add other aggregations if needed (e.g. business name, date by min or max)
    })
)

# Step 5: Count number of charges per inspection
charge_counts = charges_data["Inspection Number"].value_counts().rename("Charge Count")

# Step 6: Merge charge counts into aggregated failed inspections
merged = failed_inspections_agg.merge(charge_counts, how="left", left_on="Inspection Number", right_index=True)

# Step 7: Fill missing charge counts (no charges filed)
merged["Charge Count"] = merged["Charge Count"].fillna(0).astype(int)

# Step 8: Add Charged flag (True if Charge Count > 0)
merged["Charged"] = merged["Charge Count"] > 0

# Step 9: Analyze results
charged_count = merged["Charged"].sum()
total_failed = len(merged)

print(f"{charged_count} of {total_failed} failed inspections resulted in a formal charge.")

# Optional: Print distribution of charge counts
print("\nCharge count distribution:")
print(merged["Charge Count"].value_counts().sort_index())

# Step 10: Filter to only failed inspections that resulted in formal charges
true_charges = merged[merged["Charged"]]

# View the first few rows of the true charges dataset
print(true_charges.head())

# ✅ Step 9: Pull full company info for inspections that were charged
charged_inspection_ids = merged.loc[merged["Charged"], "Inspection Number"]

# Filter original failed_inspections (with full business info) to those IDs
full_info_charged = failed_inspections[failed_inspections["Inspection Number"].isin(charged_inspection_ids)]

# Step 10: Export full dataset of failed inspections that led to charges
full_info_charged.to_csv("Full_Failed_Inspections_With_Charges.csv", index=False)


2 of 1391 failed inspections resulted in a formal charge.

Charge count distribution:
Charge Count
0    1389
1       1
2       1
Name: count, dtype: int64
    Inspection Number  Devices Condemned/Confiscated  Charge Count  Charged
354     INS-000695112                              4             2     True
361     INS-000696027                              2             1     True


In [79]:
## Now pull the full merged dataset
full_info_all = failed_inspections.merge(
    merged[["Inspection Number", "Charge Count", "Charged"]],
    on="Inspection Number",
    how="left"
)

# Step 12: Export full dataset of failed inspections with charge info
full_info_all.to_csv("All_Failed_Inspections_Charges_Flagged.csv", index=False)